In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1
    
seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

xTrain=np.load("../extracted_features/Change/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/Change/labels_train_grp3.npy")

xTest=np.load("../extracted_features/Change/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/Change/labels_test_grp3.npy")


xVal=np.load("../extracted_features/Change/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/Change/labels_val_grp3.npy")

# train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
# labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

# test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
# labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

# val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
# labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")



In [2]:
# print(labels_train.shape,labels_train)

In [3]:
# print(train_features.shape,labels_train.shape)
# print(test_features.shape,labels_test.shape)
# print(val_features.shape,labels_val.shape)

In [4]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 8) (512,)
(122, 19, 8) (122,)
(118, 19, 8) (118,)


In [5]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [6]:
batch_size=20

train_data = Dataset(xTrain, yTrain)
test_data=Dataset(xTest,yTest)
val_data=Dataset(xVal,yVal)

# train_data = Dataset(train_features, labels_train)
# test_data=Dataset(test_features,labels_test)
# val_data=Dataset(val_features,labels_val)

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [7]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [8]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [9]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 8
        self.hidden_dim = 128
        self.layer_dim = 2
        self.output_dim=1
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [10]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 200
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [11]:
from sklearn.metrics import accuracy_score
best_acc1 = 0
modelname=[]
truth=[]
preds=[]
model.train()
for it in range(epoch+1):
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
#     model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)

EPOCH  0
Train : Loss: 0.0131, Train acc1 : 0.5586
Val : Loss: 0.0116, Val acc1 : 0.5738
Best validation accuracy1  0.5737704918032787
EPOCH  1
Train : Loss: 0.0120, Train acc1 : 0.5915
Val : Loss: 0.0123, Val acc1 : 0.5738
Best validation accuracy1  0.5737704918032787
EPOCH  2
Train : Loss: 0.0119, Train acc1 : 0.5994
Val : Loss: 0.0110, Val acc1 : 0.5902
Best validation accuracy1  0.5901639344262295
EPOCH  3
Train : Loss: 0.0117, Train acc1 : 0.6041
Val : Loss: 0.0118, Val acc1 : 0.5902
Best validation accuracy1  0.5901639344262295
EPOCH  4
Train : Loss: 0.0115, Train acc1 : 0.6151
Val : Loss: 0.0115, Val acc1 : 0.5492
EPOCH  5
Train : Loss: 0.0113, Train acc1 : 0.6120
Val : Loss: 0.0122, Val acc1 : 0.5656
EPOCH  6
Train : Loss: 0.0114, Train acc1 : 0.6199
Val : Loss: 0.0116, Val acc1 : 0.5656
EPOCH  7
Train : Loss: 0.0111, Train acc1 : 0.6341
Val : Loss: 0.0117, Val acc1 : 0.5656
EPOCH  8
Train : Loss: 0.0111, Train acc1 : 0.6246
Val : Loss: 0.0115, Val acc1 : 0.5164
EPOCH  9
Train 

EPOCH  84
Train : Loss: 0.0058, Train acc1 : 0.8091
Val : Loss: 0.0135, Val acc1 : 0.6230
EPOCH  85
Train : Loss: 0.0052, Train acc1 : 0.8233
Val : Loss: 0.0110, Val acc1 : 0.6557
EPOCH  86
Train : Loss: 0.0052, Train acc1 : 0.8202
Val : Loss: 0.0117, Val acc1 : 0.6721
Best validation accuracy1  0.6721311475409836
EPOCH  87
Train : Loss: 0.0050, Train acc1 : 0.8360
Val : Loss: 0.0142, Val acc1 : 0.6721
Best validation accuracy1  0.6721311475409836
EPOCH  88
Train : Loss: 0.0053, Train acc1 : 0.8281
Val : Loss: 0.0122, Val acc1 : 0.6475
EPOCH  89
Train : Loss: 0.0049, Train acc1 : 0.8360
Val : Loss: 0.0109, Val acc1 : 0.6639
EPOCH  90
Train : Loss: 0.0052, Train acc1 : 0.8202
Val : Loss: 0.0123, Val acc1 : 0.6721
Best validation accuracy1  0.6721311475409836
EPOCH  91
Train : Loss: 0.0048, Train acc1 : 0.8391
Val : Loss: 0.0137, Val acc1 : 0.6475
EPOCH  92
Train : Loss: 0.0052, Train acc1 : 0.8202
Val : Loss: 0.0121, Val acc1 : 0.6393
EPOCH  93
Train : Loss: 0.0048, Train acc1 : 0.8312


EPOCH  171
Train : Loss: 0.0014, Train acc1 : 0.9338
Val : Loss: 0.0113, Val acc1 : 0.6967
EPOCH  172
Train : Loss: 0.0018, Train acc1 : 0.9274
Val : Loss: 0.0107, Val acc1 : 0.6803
EPOCH  173
Train : Loss: 0.0013, Train acc1 : 0.9259
Val : Loss: 0.0108, Val acc1 : 0.6967
EPOCH  174
Train : Loss: 0.0016, Train acc1 : 0.9290
Val : Loss: 0.0116, Val acc1 : 0.6967
EPOCH  175
Train : Loss: 0.0013, Train acc1 : 0.9322
Val : Loss: 0.0131, Val acc1 : 0.7049
EPOCH  176
Train : Loss: 0.0017, Train acc1 : 0.9322
Val : Loss: 0.0108, Val acc1 : 0.6967
EPOCH  177
Train : Loss: 0.0012, Train acc1 : 0.9338
Val : Loss: 0.0111, Val acc1 : 0.6885
EPOCH  178
Train : Loss: 0.0015, Train acc1 : 0.9306
Val : Loss: 0.0110, Val acc1 : 0.6803
EPOCH  179
Train : Loss: 0.0012, Train acc1 : 0.9322
Val : Loss: 0.0110, Val acc1 : 0.6803
EPOCH  180
Train : Loss: 0.0015, Train acc1 : 0.9243
Val : Loss: 0.0115, Val acc1 : 0.6885
EPOCH  181
Train : Loss: 0.0011, Train acc1 : 0.9322
Val : Loss: 0.0115, Val acc1 : 0.6803

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.47 0.36 0.44 0.31 0.46 0.62


In [13]:
xTest=np.load("../extracted_features/Change1/input_features_val_grp3.npy")
yTest=np.load("../extracted_features/Change1/labels_val_grp3.npy")

# xTest = xTest.reshape(len(xTest),1,19,24)

testdataset = Dataset(xTest,yTest)

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.41 0.29 0.35 0.24 0.40 0.57
